In [ ]:
#Pandas, Plotly, and NumPy modules
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd

# Modules imported to retrieve data from Google Sheets
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

SPREADSHEET_ID = '1-kUlYLeDEQyzw2PYnLn6quq7kFYJbduI_Zrk_bQ7_9A'
RANGE_NAME = 'PlayerOllie'

def layout():
    
    return html.Div(className='body', children=[
                html.Div(className='title', children=[
                    html.H1('James Knowlton'),
                    html.H2('Player Report')
                ]),
                html.Div(className='stats', children=[
                    html.Div(className='offense', children=[
                        html.H2('Offense'),
                        html.Ul(children=[
                            html.Li('Goals: '),
                            html.Li('Shots: '),
                            html.Li('Shots on Goal: '),
                            html.Li('Assists: '),
                            html.Li('Passes: '),
                            html.Li('Success Rate with Ball: '),
                            html.Li('Turnovers: ')
                        ], style={
                            'listStyle':'none',
                            'padding':0
                        })
                    ]),
                    html.Div(className='defense', children=[
                        html.H2('Defense'),
                        html.Ul(children=[
                            html.Li('Fouls: '),
                            html.Li('Recoveries: '),
                            html.Li('Yellow Cards: '),
                            html.Li('Red Cards: ')
                        ], style={
                            'listStyle':'none',
                            'padding':0
                        })
                    ])
                ], style={
                    'display':'flex',
                    'gap':20
                }),
                html.Div(className='radial-chart', children=[
                    html.H2('Insert Radial Here')
                    
                ]),
                html.Div(className='position', children=[
                    html.P('Insert Position Here')
                ]),
                html.Div(className='line-chart', children=[
                    html.P('Insert Line Here')
                ])
            ])

app = dash.Dash(__name__)

app.layout = layout()

if __name__ == '__main__':
    app.run_server(debug=False)